In [1214]:
import xlutils
import databaker
import databaker.constants
from databaker.constants import *
import xypath
import databaker.databakersolo as ds
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup

from databaker.jupybakeutils import dsubsets, tabletohtml, sidewindowhtmldisplay, inlinehtmldisplay

from databaker.structure_csv_default import dimension_names, SKIP_AFTER
    

In [1215]:
tableset = xypath.loader.table_set("ilchtablestemplatensa.xls", extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, ['1. Industry level', '2. Sector level', '3. Industry growth rates', '4. Sector growth Rates']))


In [1216]:

"""
Not the full thing, I've captured 2 basic dimensions only so i could skip straight to post processing
"""

for tab in tabs:
    tab_title = tab.excel_ref("A1")

    anchor = tab.filter("Period").assert_one()
        
    # Pre selction
    datarows = anchor.fill(DOWN).is_not_blank()
    datacols = anchor.shift(DOWN).fill(RIGHT).is_not_blank()
    obs = datarows.waffle(datacols).is_not_blank()
        
    # Assign Consistent Dimensions
    dimensions = [
                (datarows, TIME, DIRECTLY, LEFT),
                (datacols.parent(), "Costs", DIRECTLY, ABOVE),
                 ]
    
    conversionsegments = []
    conversionsegments.append((tab, dimensions, obs))


In [1217]:

""" Wrappers """
""" Trying out, putting dimensions into an object with post-processing as methods"""
class post(object):
    
    def __init__(self, tab, dimensions, segment):
        self.tab = tab
        self.dimensions = dimensions
        self.segment = segment
        self.choice = ''
        self.static = [x[1] for x in dimensions if type(x[1]) == int]
        self.topic = [x[1] for x in dimensions if type(x[1]) == str]
        self.static_as_name = [(x, dimension_names[x]) for x in self.static]
    
    
    """ Print a list of dims"""
    def listdims(self):
        dimlist = []
        for each in self.static_as_name:
            dimlist.append(each[1])
        for each in self.topic:
            dimlist.append(each)
        print ('Dimensions:', dimlist)
        
        
    """ Get requested dimension based on name provided (so somthing from listdims)"""
    def getDim(self, dim):
    
        # Get any static dimensions back into negative index (i.e obs = -9 etc)
        if dim in [x[1] for x in self.static_as_name]:
            dim = ([x[0] for x in self.static_as_name if x[1] == dim])[0]
        
        # Get the correct dimension from self.dimensions
        dimension = [x for x in self.dimensions if x[1] == dim][0]
        return dimension, dim
    
    
    """ Get a tuple of just values from a dimension """
    def unbuild(self, dimension):
        all_values = ()
        for item in dimension[0]:
            all_values = all_values + (item.value,)
        return all_values    
        
    
    """ Get new values into the right place in a new bag, then replace original bag """
    def rebuild(self, dimIndex, dimension, all_values):
        """
        Build new xyCells with the new value (where applicable)
        Put them into a new bag
        Put that bag into a dimension
        Put that dimension into dimensions (note pural) in self.
        """     
        allValues = []
        for i, item in enumerate(dimension[0]):
            
            # borrow _XYCell object, rebuild with new value and everything we had before
            # not sure if this is circular, but saves a lot of screwing around
            newcell = xypath.xypath._XYCell(all_values[i], item.x, item.y, item.table, properties=item.properties)
            allValues.append(newcell)

        # build a new bag
        newBag = xypath.xypath.Bag(none)   # none argument for table, otherwise it complains
        for val in allValues:
            newBag.add(val)

        # rebuild dimension tuple with this new bag
        new_dimension = (newBag, dimension[1], dimension[2], dimension[3])
        
        # rebuild self.dimensions with this modified dimension
        new_dimensions = []
        for oldDimension in dimensions:
            if oldDimension[1] == dimIndex:
                new_dimensions.append(new_dimension)
            else:
                new_dimensions.append(oldDimension)

        # set post-processed dimensions
        self.dimensions = new_dimensions

    """ Removes everything on the list of things to be removed (unwanted)"""
    def remove(self, unwanted):
        # just gonna piggyback replace
        replace = {}
        for unwant in unwanted:
            replace.update({unwant:''})
        self.replace(replace)
        return self
        
    """ Replace text using a dictionary of key:value = old:new (replace)"""
    def replace(self, replace):       
        dimension, dimIndex = self.getDim(self.choice)
        oldValues = self.unbuild(dimension)
        
        newValues = []
        for value in oldValues:
            for unwanted in replace.keys():
                if unwanted in value:
                     value = value.replace(unwanted, replace[unwanted])
            newValues.append(value)
        
        self.rebuild(dimIndex, dimension, newValues)
        return self
    
    """ simple string.strip() on values """
    def clean(self):
        dimension, dimIndex = self.getDim(self.choice)
        oldValues = self.unbuild(dimension)

        newValues = []
        for value in oldValues:
            value = value.strip()
            newValues.append(value)
        
        self.rebuild(dimIndex, dimension, newValues)
        return self

    
    """ Adds some text at the specified index in value string """
    def insert(self, i, txt):
        # TODO - needs an error message if out of index range
        dimension, dimIndex = self.getDim(self.choice)
        oldValues = self.unbuild(dimension)

        newValues = []
        for value in oldValues:
            value = value[:i] + txt + value[i:]
            newValues.append(value)
        
        self.rebuild(dimIndex, dimension, newValues)
        return self
    
    """ Set which dimension we're editing """
    def edit(self, dim):
        self.choice = dim
        return self




In [1218]:

# using hacky wrapper


# load into object
data = post(tab, dimensions, segment)

# see what we start with
print ('Before', data.dimensions)
print ('')

data.listdims()

# strip whitespace, replace 20 with 30, remove Q1,2,3,4 text, put 'Start-' at the beginning
data.edit("TIMEUNIT").clean().replace({"20":"30"}).remove(["Q1", "Q2", "Q3", "Q4"]).insert(0, 'Start-')

"""
NOTE
Some of this (insert in particular) could do with being triggered conditionally, not sure how that'd look
Not sure how sorting would come into it either as that seems to be in a different logical place...i think
"""

# see what we have now
print ('')
print ('After', data.dimensions)
print ('')



Before [({<A25 '2003Q3'>, <A62 '2012Q4 '>, <A28 '2004Q2'>, <A49 '2009Q3'>, <A64 '2013Q2'>, <A79 'r = Revised'>, <A63 '2013Q1'>, <A13 '2000Q3'>, <A73 '2015Q3 (r)'>, <A20 '2002Q2'>, <A34 '2005Q4'>, <A31 '2005Q1'>, <A21 '2002Q3'>, <A53 '2010Q3'>, <A37 '2006Q3'>, <A52 '2010Q2'>, <A45 '2008Q3'>, <A65 '2013Q3'>, <A35 '2006Q1'>, <A69 '2014Q3 (r)'>, <A51 '2010Q1'>, <A42 '2007Q4'>, <A33 '2005Q3'>, <A57 '2011Q3'>, <A41 '2007Q3'>, <A76 '2016Q2 (p)'>, <A23 '2003Q1'>, <A18 '2001Q4'>, <A72 '2015Q2 (r)'>, <A58 '2011Q4'>, <A22 '2002Q4'>, <A70 '2014Q4 (r)'>, <A74 '2015Q4 (r)'>, <A19 '2002Q1'>, <A82 'ILCH enquiries: Email: ster@ons.gsi.gov.uk'>, <A30 '2004Q4'>, <A43 '2008Q1'>, <A71 '2015Q1 (r)'>, <A78 'p = Provisional'>, <A55 '2011Q1'>, <A59 '2012Q1 '>, <A66 '2013Q4'>, <A48 '2009Q2'>, <A54 '2010Q4'>, <A75 '2016Q1 (r)'>, <A29 '2004Q3'>, <A16 '2001Q2'>, <A12 '2000Q2'>, <A15 '2001Q1'>, <A39 '2007Q1'>, <A47 '2009Q1'>, <A81 'Source: Monthly Wages and Salaries Survey, Labour Force Survey'>, <A56 '2011Q2'>, <A